데이터 출처
- Kaggle : The movies Dataset (https://www.kaggle.com/rounakbanik/the-movies-dataset)

해당 자료는 아래 링크에서 참조했습니다.
- https://lsjsj92.tistory.com/565

# 데이터 전처리

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

사용할 데이터 목록
- movie_metadata.csv : 제목과 장르 등의 영화 메타 데이터
- keywords.csv : 영화 id에 따라 keyword 값

사용할 칼럼 목록
- genres : 영화 장르
- keywords : 영화의 키워드
- original_language : 영화 언어
- title : 제목
- vote_average : 평점 평균
- vote_count : 평점 카운트
- popularity : 인기도
- overview : 개요 설명

## df1

In [35]:
df1 = pd.read_csv('/content/drive/Othercomputers/내 PC/DEV/DSL/심화스터디/추천시스템/movie_data/movies_metadata.csv')
df1 = df1[['id', 'title', 'original_language', 'genres', 'popularity', 'vote_average', 'vote_count', 'overview']]

print(df1.shape)
df1.head()

(45466, 8)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,title,original_language,genres,popularity,vote_average,vote_count,overview
0,862,Toy Story,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",21.946943,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ..."
1,8844,Jumanji,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",17.015539,6.9,2413.0,When siblings Judy and Peter discover an encha...
2,15602,Grumpier Old Men,en,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",11.7129,6.5,92.0,A family wedding reignites the ancient feud be...
3,31357,Waiting to Exhale,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",3.859495,6.1,34.0,"Cheated on, mistreated and stepped on, the wom..."
4,11862,Father of the Bride Part II,en,"[{'id': 35, 'name': 'Comedy'}]",8.387519,5.7,173.0,Just when George Banks has recovered from his ...


## score 보정
- 식 참고 : https://www.quora.com/How-does-IMDbs-rating-system-work
- r : 개별 영화 평점
- v : 개별 영화에 평점을 투표한 횟수
- m : 250위 안에 들어야 하는 최소 투표 수
- c : 전체 영화에 대한 평균 평점

-> 일단 m값을 정하자!

In [46]:
tmp_m = df1['vote_count'].quantile(0.9)
tmp_m

160.0

In [49]:
df1 = df1.loc[df1['vote_count'] >= tmp_m]
df1.shape

(4555, 8)

- 투표수가 90% 값 이상 되도록 m 설정 -> 4555개 얻어짐

In [50]:
df1.head()

,id,title,original_language,genres,popularity,vote_average,vote_count,overview
0,862,Toy Story,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",21.946943,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ..."
1,8844,Jumanji,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",17.015539,6.9,2413.0,When siblings Judy and Peter discover an encha...
4,11862,Father of the Bride Part II,en,"[{'id': 35, 'name': 'Comedy'}]",8.387519,5.7,173.0,Just when George Banks has recovered from his ...
5,949,Heat,en,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",17.924927,7.7,1886.0,"Obsessive master thief, Neil McCauley leads a ..."
8,9091,Sudden Death,en,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",5.23158,5.5,174.0,International action superstar Jean Claude Van...


In [51]:
C = df1['vote_average'].mean()
m = tmp_m
print(C)
print(m)

6.473589462129529
160.0


In [52]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    
    return ( v / (v+m) * R ) + (m / (m + v) * C)

In [53]:
df1['score'] = df1.apply(weighted_rating, axis = 1)
df1.head()

,id,title,original_language,genres,popularity,vote_average,vote_count,overview,score
0,862,Toy Story,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",21.946943,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ...",7.664803
1,8844,Jumanji,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",17.015539,6.9,2413.0,When siblings Judy and Peter discover an encha...,6.873484
4,11862,Father of the Bride Part II,en,"[{'id': 35, 'name': 'Comedy'}]",8.387519,5.7,173.0,Just when George Banks has recovered from his ...,6.071695
5,949,Heat,en,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",17.924927,7.7,1886.0,"Obsessive master thief, Neil McCauley leads a ...",7.604093
8,9091,Sudden Death,en,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",5.23158,5.5,174.0,International action superstar Jean Claude Van...,5.966390


In [73]:
df1 = df1.drop_duplicates(['id'])
df1.shape

(4551, 9)

## df2 (keyword)

In [55]:
df2 = pd.read_csv('/content/drive/Othercomputers/내 PC/DEV/DSL/심화스터디/추천시스템/movie_data/keywords.csv')
print(df2.shape)
df2.head()

(46419, 2)


,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [74]:
# id 중복행 제거
df2 = df2.drop_duplicates(['id'])
df2.shape

(45432, 2)

두 데이터셋을 id에 따라 merge하자

In [76]:
 # 일단 int로
df1.id = df1.id.astype(int)
df2.id = df2.id.astype(int)

data = pd.merge(df1, df2, left_on='id', right_on='id', how='inner')
print(data.shape)
data.head()

(4551, 10)


,id,title,original_language,genres,popularity,vote_average,vote_count,overview,score,keywords
0,862,Toy Story,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",21.946943,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ...",7.664803,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,Jumanji,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",17.015539,6.9,2413.0,When siblings Judy and Peter discover an encha...,6.873484,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,11862,Father of the Bride Part II,en,"[{'id': 35, 'name': 'Comedy'}]",8.387519,5.7,173.0,Just when George Banks has recovered from his ...,6.071695,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
3,949,Heat,en,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",17.924927,7.7,1886.0,"Obsessive master thief, Neil McCauley leads a ...",7.604093,"[{'id': 642, 'name': 'robbery'}, {'id': 703, '..."
4,9091,Sudden Death,en,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",5.23158,5.5,174.0,International action superstar Jean Claude Van...,5.966390,"[{'id': 949, 'name': 'terrorist'}, {'id': 1562..."


## genres, keywords 보정

In [77]:
data['genres'] = data['genres'].apply(literal_eval)
data['keywords'] = data['keywords'].apply(literal_eval)
data.head()

,id,title,original_language,genres,popularity,vote_average,vote_count,overview,score,keywords
0,862,Toy Story,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",21.946943,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ...",7.664803,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,Jumanji,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",17.015539,6.9,2413.0,When siblings Judy and Peter discover an encha...,6.873484,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,11862,Father of the Bride Part II,en,"[{'id': 35, 'name': 'Comedy'}]",8.387519,5.7,173.0,Just when George Banks has recovered from his ...,6.071695,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
3,949,Heat,en,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",17.924927,7.7,1886.0,"Obsessive master thief, Neil McCauley leads a ...",7.604093,"[{'id': 642, 'name': 'robbery'}, {'id': 703, '..."
4,9091,Sudden Death,en,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",5.23158,5.5,174.0,International action superstar Jean Claude Van...,5.966390,"[{'id': 949, 'name': 'terrorist'}, {'id': 1562..."


In [78]:
data['genres'] = data['genres'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))
data['keywords'] = data['keywords'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))
data.head()

,id,title,original_language,genres,popularity,vote_average,vote_count,overview,score,keywords
0,862,Toy Story,en,Animation Comedy Family,21.946943,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ...",7.664803,jealousy toy boy friendship friends rivalry bo...
1,8844,Jumanji,en,Adventure Fantasy Family,17.015539,6.9,2413.0,When siblings Judy and Peter discover an encha...,6.873484,board game disappearance based on children's b...
2,11862,Father of the Bride Part II,en,Comedy,8.387519,5.7,173.0,Just when George Banks has recovered from his ...,6.071695,baby midlife crisis confidence aging daughter ...
3,949,Heat,en,Action Crime Drama Thriller,17.924927,7.7,1886.0,"Obsessive master thief, Neil McCauley leads a ...",7.604093,robbery detective bank obsession chase shootin...
4,9091,Sudden Death,en,Action Adventure Thriller,5.23158,5.5,174.0,International action superstar Jean Claude Van...,5.966390,terrorist hostage explosive vice president


# Content Based
- 비슷한 영화를 추천해주는 것이므로 genre와 keyword를 활용하자

## TfidfVectorizer
- Converting text data into vectors as model can process only numerical data
- vs CountVectorizer : https://www.quora.com/What-is-the-difference-between-TfidfVectorizer-and-CountVectorizer-1


In [80]:
tfidf_vector = TfidfVectorizer()
tfidf_matrix = tfidf_vector.fit_transform(data['genres'] + " " + data['keywords']).toarray()

tfidf_matrix_feature = tfidf_vector.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [81]:
tfidf_matrix.shape

(4551, 7507)

In [84]:
tfidf_matrix = pd.DataFrame(tfidf_matrix, columns=tfidf_matrix_feature, index = data.title)
tfidf_matrix.head()

,11,15th,16th,17th,18th,1905,1910s,1917,1920s,1930s,...,zookeeper,zoom,zoophilia,zumaia,zurich,øverste,γη,卧底肥妈,绝地奶霸,超级妈妈
title,,,,,,,,,,,,,,,,,,,,,
Toy Story,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jumanji,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Father of the Bride Part II,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Heat,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Sudden Death,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 유사도 구하기
- tf-idf vector 활용해 코사인 유사도 계산

In [85]:
cosine_sim = cosine_similarity(tfidf_matrix)
cosine_sim.shape

(4551, 4551)

In [87]:
cosine_sim_df = pd.DataFrame(cosine_sim, index = data.title, columns = data.title)
cosine_sim_df.head()

title,Toy Story,Jumanji,Father of the Bride Part II,Heat,Sudden Death,GoldenEye,The American President,Dracula: Dead and Loving It,Balto,Casino,...,"La leggenda di Al, John e Jack",2:22,The Dark Tower,The Emoji Movie,Wind River,Shot Caller,Brice 3,Girls Trip,Descendants 2,The Visitors: Bastille Day
title,,,,,,,,,,,,,,,,,,,,,
Toy Story,1.000000,0.039716,0.006777,0.000000,0.000000,0.000000,0.036515,0.015014,0.026214,0.000000,...,0.073776,0.000000,0.000000,0.056620,0.000000,0.000000,0.073776,0.073776,0.024343,0.010779
Jumanji,0.039716,1.000000,0.000000,0.000000,0.022505,0.009312,0.038356,0.000000,0.026221,0.000000,...,0.000000,0.000000,0.113765,0.025263,0.000000,0.000000,0.000000,0.000000,0.040226,0.000000
Father of the Bride Part II,0.006777,0.000000,1.000000,0.000000,0.000000,0.000000,0.010167,0.018694,0.000000,0.000000,...,0.091857,0.000000,0.000000,0.012100,0.000000,0.000000,0.091857,0.091857,0.010375,0.013421
Heat,0.000000,0.000000,0.000000,1.000000,0.018271,0.007560,0.004841,0.000000,0.000000,0.030143,...,0.000000,0.076091,0.012064,0.000000,0.069073,0.225840,0.000000,0.000000,0.074859,0.000000
Sudden Death,0.000000,0.022505,0.000000,0.018271,1.000000,0.031641,0.117928,0.000000,0.016600,0.000000,...,0.000000,0.111179,0.030105,0.000000,0.027461,0.017553,0.000000,0.000000,0.045663,0.000000


## 추천 함수

기본 설명
- target title (추천 결과를 조회할 영화 제목)에 따라서 코사인 유사도를 구한 matrix에서 유사도 데이터를 가져옴
- 유사도 데이터 중 가장 유사도 값이 큰 데이터를 가져옴 : 가져올 때 top k개를 가져옴
- 해당 추천 값 출력

In [119]:
def genre_recommendations(target_title, matrix, items, k=10):
    recom_idx = matrix.loc[:, target_title].values.reshape(1, -1).argsort()[:, ::-1].flatten()[1:k+1]
    ret = items.iloc[recom_idx, :]
    return ret[['title','score','genres','keywords']]

In [120]:
genre_recommendations('The Dark Knight Rises', cosine_sim_df, data)

,title,score,genres,keywords
2397,The Dark Knight,8.276488,Drama Action Crime Thriller,dc comics crime fighter secret identity scarec...
1942,Batman Begins,7.478591,Action Crime Drama,himalaya martial arts dc comics crime fighter ...
483,Batman & Robin,4.426369,Action Crime Fantasy,double life dc comics dual identity crime figh...
185,Batman,6.963460,Fantasy Action,double life dc comics dual identity chemical c...
2367,Cleaner,6.154222,Crime Thriller Mystery,cover-up
444,Batman Returns,6.589161,Action Fantasy,holiday corruption double life dc comics crime...
4013,Batman vs. Robin,6.661837,Action Adventure Animation,dc comics gotham city based on comic robin sup...
2771,Defendor,6.488163,Drama Action Comedy Crime,crime fighter delusion superhero
4480,Justice League Dark,6.745542,Action Animation Fantasy Science Fiction Thriller,dc comics monster supernatural superhero based...
1769,Catwoman,4.566339,Action Crime,white russian sex dc comics beauty sexism bask...


In [129]:
genre_recommendations('Pulp Fiction', cosine_sim_df, data)

,title,score,genres,keywords
1726,Mean Streets,6.976058,Drama Crime,epilepsy protection money secret love money re...
752,Psycho,5.481360,Horror Mystery Thriller,motel stolen money
577,The French Connection,7.150881,Action Crime Thriller,police brutality marseille night life attempte...
1343,Ali,6.641287,Drama,usa transporter boxer biography muhammad
1845,Pusher,6.637808,Action Crime Drama Thriller,copenhagen drug dealer police operation drug
3754,Step Up All In,6.655222,Romance Drama Music,street artist dance contest woman director
2937,Unknown,6.497082,Action Mystery Thriller,taxi hotel coma taxi driver prince briefcase h...
3264,Compliance,6.372141,Drama Thriller Crime,
3836,Zulu,6.592185,Crime Drama Thriller,
2836,Animal Kingdom,6.609436,Drama Thriller Crime,


## 확장해볼 거리
- 원래 (강의때 배운 바로는) user가 좋아하는 item들을 처음에 따로 모으는 과정 필요 -> 그 item들 토대로 user profile 구성
- user profile 기반 유사한 item 추천 (여기서는 item 바로 입력해서 유사한 item 추천)
- genres, keywords 이외의 다른 특성 고려할 수도 있음